In [1]:
# Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!/usr/bin/env python3
"""
FINAL RECOMMENDED PIPELINE (Code4 + Best Diagnostics from Code3)
===============================================================

Ecuador Climate Data Cleaning for EVT (Google Colab)

What this script does
---------------------
✅ EVT-safe cleaning (NO outlier removal, NO imputation)
✅ Memory-safe ingestion (chunked)
✅ Robust datetime parsing for dt_iso strings like:
      '1979-01-01 00:00:00 +0000 UTC'
✅ Local time conversion using row-wise timezone offset (seconds)
✅ Stores fecha_local as NAIVE local clock time (tz removed) to avoid misleading "+00:00"
✅ Fixes city name typo: 'Lago Agrío' -> 'Lago Agrio' (removes Spanish tilde)
✅ Drops only globally 100% null columns (detected, not hard-coded)
✅ Optimizes dtypes: float32, downcast ints, categories for low-cardinality strings
✅ Sort output by city_name, fecha_local (global) using:
      chunk -> per-city parts -> per-city sort -> merge
✅ Exports Parquet (Snappy): data_clima_clean.parquet

Diagnostics added (best parts of Code3, actionable not spammy)
--------------------------------------------------------------
- Parse failure count
- Timezone mismatch count (fecha_local shift check)
- Global missingness summary (top columns)
- Per-city row counts
- Per-city timezone distribution (counts and percentages)
- Duplicate timestamps per city (counts + rate) based on fecha_local
- Max time gap per city (days) in fecha_local
- Basic EVT preview for selected variables (min/max/quantiles) WITHOUT filtering

Paths
-----
Input : /content/drive/MyDrive/extreme-climate-forecasting/data/data_clima.csv
Output: /content/drive/MyDrive/extreme-climate-forecasting/data/data_clima_clean.parquet

Temp dirs:
  /content/drive/MyDrive/extreme-climate-forecasting/data/_tmp_evt_city_parts_final
  /content/drive/MyDrive/extreme-climate-forecasting/data/_tmp_evt_city_sorted_final
"""

from __future__ import annotations

import gc
import shutil
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq


# =============================================================================
# CONFIG
# =============================================================================
DATA_DIR = Path("/content/drive/MyDrive/extreme-climate-forecasting/data/")
CSV_PATH = DATA_DIR / "data_clima.csv"
OUT_PARQUET = DATA_DIR / "data_clima_clean.parquet"

TMP_PARTS_DIR = DATA_DIR / "_tmp_evt_city_parts_final"
TMP_SORTED_DIR = DATA_DIR / "_tmp_evt_city_sorted_final"

CHUNKSIZE = 300_000  # reduce if RAM errors

CATEGORICAL_COLS = ["city_name", "weather_main", "weather_description", "weather_icon"]

# Variables to summarize for EVT preview (non-destructive)
EVT_PREVIEW_FLOATS = ["temp", "rain_1h", "rain_3h", "wind_gust", "wind_speed"]
EVT_PREVIEW_Q = [0.95, 0.99]


# =============================================================================
# HELPERS
# =============================================================================
def ensure_clean_dir(path: Path) -> None:
    if path.exists():
        shutil.rmtree(path)
    path.mkdir(parents=True, exist_ok=True)


def parse_dt_iso_to_utc(dt_iso: pd.Series) -> pd.Series:
    """
    Parse dt_iso strings like:
      '1979-01-01 00:00:00 +0000 UTC'
    into timezone-aware UTC datetimes.

    We strip trailing ' UTC' and parse with explicit format for speed & consistency.
    """
    s = dt_iso.astype("string").str.replace(" UTC", "", regex=False)
    return pd.to_datetime(
        s,
        format="%Y-%m-%d %H:%M:%S %z",
        errors="coerce",
        utc=True,
    )


def optimize_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """Downcast numeric types and set low-cardinality objects to category (no rounding)."""
    # float64 -> float32
    for c in df.select_dtypes(include=["float64"]).columns:
        df[c] = df[c].astype("float32")

    # int64 -> smallest safe int
    for c in df.select_dtypes(include=["int64"]).columns:
        df[c] = pd.to_numeric(df[c], downcast="integer")

    # categories
    for c in CATEGORICAL_COLS:
        if c in df.columns:
            df[c] = df[c].astype("category")

    return df


def reorder_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Identifiers/time -> location -> remaining columns."""
    id_time = [
        "city_name",
        "dt",
        "dt_iso",
        "dt_utc",
        "timezone",
        "fecha_local",
        "year",
        "month",
        "day",
        "hour",
    ]
    location = ["lat", "lon"]

    ordered = [c for c in id_time if c in df.columns] + [c for c in location if c in df.columns]
    remaining = [c for c in df.columns if c not in set(ordered)]
    return df[ordered + remaining]


def update_null_stats(df: pd.DataFrame, null_counts: Dict[str, int], nonnull_counts: Dict[str, int]) -> None:
    n = len(df)
    for c in df.columns:
        miss = int(df[c].isna().sum())
        null_counts[c] = null_counts.get(c, 0) + miss
        nonnull_counts[c] = nonnull_counts.get(c, 0) + (n - miss)


def update_counts(counts: Dict[str, int], series: pd.Series) -> None:
    vc = series.value_counts(dropna=False)
    for k, v in vc.items():
        key = "__NA__" if pd.isna(k) else str(k)
        counts[key] = counts.get(key, 0) + int(v)


def write_city_parts(chunk: pd.DataFrame, base_dir: Path, part_counter: Dict[str, int]) -> None:
    """
    Write a chunk into per-city parquet parts:
      base_dir/city_name=Quito/part-000001.parquet
    """
    city_series = chunk["city_name"].astype("string")

    for city in city_series.unique():
        if pd.isna(city):
            city_key = "__NA__"
            mask = city_series.isna()
        else:
            city_key = str(city)
            mask = (city_series == city_key)

        df_city = chunk.loc[mask].copy()

        out_dir = base_dir / f"city_name={city_key}"
        out_dir.mkdir(parents=True, exist_ok=True)

        part_counter.setdefault(city_key, 0)
        part_counter[city_key] += 1
        out_file = out_dir / f"part-{part_counter[city_key]:06d}.parquet"

        pq.write_table(pa.Table.from_pandas(df_city, preserve_index=False), out_file, compression="snappy")
        del df_city

    del city_series
    gc.collect()


def safe_quantiles(series: pd.Series, q_list: List[float]) -> Dict[float, float]:
    """Compute quantiles on non-null data; return empty if no data."""
    s = series.dropna()
    if len(s) == 0:
        return {}
    return {q: float(s.quantile(q)) for q in q_list}


# =============================================================================
# MAIN PIPELINE
# =============================================================================
def clean_evt_pipeline(csv_path: Path, out_parquet: Path, chunksize: int = 300_000) -> None:
    assert csv_path.exists(), f"CSV not found: {csv_path}"

    ensure_clean_dir(TMP_PARTS_DIR)
    ensure_clean_dir(TMP_SORTED_DIR)

    # Global counters/stats
    total_rows_in = 0
    total_rows_out = 0

    null_counts: Dict[str, int] = {}
    nonnull_counts: Dict[str, int] = {}

    dt_parse_fail_count = 0
    tz_mismatch_count = 0

    min_fecha_local: Optional[pd.Timestamp] = None
    max_fecha_local: Optional[pd.Timestamp] = None

    city_counts: Dict[str, int] = {}
    timezone_counts: Dict[str, Dict[int, int]] = {}  # city -> {tz_offset: count}

    # EVT preview aggregations (approx: keep global min/max and quantiles on a sample)
    evt_minmax: Dict[str, Tuple[Optional[float], Optional[float]]] = {v: (None, None) for v in EVT_PREVIEW_FLOATS}
    evt_sample: Dict[str, List[float]] = {v: [] for v in EVT_PREVIEW_FLOATS}
    SAMPLE_PER_CHUNK = 3000  # keep it small; used for quantiles

    part_counter: Dict[str, int] = {}

    print("=== Phase 1: Chunked transform + per-city partition write ===")
    reader = pd.read_csv(csv_path, chunksize=chunksize)

    for i, chunk in enumerate(reader, start=1):
        total_rows_in += len(chunk)

        # Required columns
        for col in ("dt_iso", "timezone", "city_name"):
            if col not in chunk.columns:
                raise ValueError(f"Missing required column: {col}")

        # Fix city name typo: 'Lago Agrío' -> 'Lago Agrio' (remove Spanish tilde)
        chunk["city_name"] = chunk["city_name"].str.replace("Lago Agrío", "Lago Agrio", regex=False)

        # Parse dt_iso -> dt_utc
        chunk["dt_utc"] = parse_dt_iso_to_utc(chunk["dt_iso"])
        bad_dt = chunk["dt_utc"].isna()
        dt_parse_fail_count += int(bad_dt.sum())

        # Shift to local time using timezone seconds, then remove tz marker -> naive local clock time
        shifted = chunk["dt_utc"] + pd.to_timedelta(chunk["timezone"], unit="s")
        chunk["fecha_local"] = shifted.dt.tz_localize(None)

        # Extract components
        chunk["year"] = chunk["fecha_local"].dt.year
        chunk["month"] = chunk["fecha_local"].dt.month
        chunk["day"] = chunk["fecha_local"].dt.day
        chunk["hour"] = chunk["fecha_local"].dt.hour

        # Timezone sanity check
        valid = chunk["dt_utc"].notna()
        delta_sec = (shifted.loc[valid] - chunk.loc[valid, "dt_utc"]).dt.total_seconds()
        tz_mismatch = (delta_sec != chunk.loc[valid, "timezone"].astype("float64"))
        tz_mismatch_count += int(tz_mismatch.sum())

        # Global date range
        fl = chunk["fecha_local"]
        if fl.notna().any():
            cmin, cmax = fl.min(), fl.max()
            min_fecha_local = cmin if (min_fecha_local is None or cmin < min_fecha_local) else min_fecha_local
            max_fecha_local = cmax if (max_fecha_local is None or cmax > max_fecha_local) else max_fecha_local

        # Null stats BEFORE dropping (so we can detect globally 100% null)
        update_null_stats(chunk, null_counts, nonnull_counts)

        # Per-city counts and timezone distribution
        update_counts(city_counts, chunk["city_name"].astype("string"))

        # timezone per city
        city_ser = chunk["city_name"].astype("string")
        tz_ser = chunk["timezone"]
        # lightweight group count per chunk
        tmp = pd.DataFrame({"city": city_ser, "tz": tz_ser})
        grp = tmp.groupby(["city", "tz"]).size()
        for (c, tz), cnt in grp.items():
            c_key = "__NA__" if pd.isna(c) else str(c)
            timezone_counts.setdefault(c_key, {})
            timezone_counts[c_key][int(tz)] = timezone_counts[c_key].get(int(tz), 0) + int(cnt)
        del tmp, grp

        # EVT preview: min/max and small reservoir sample per chunk
        for v in EVT_PREVIEW_FLOATS:
            if v in chunk.columns:
                s = chunk[v]
                # update min/max
                s_non = s.dropna()
                if len(s_non) > 0:
                    vmin, vmax = float(s_non.min()), float(s_non.max())
                    cur_min, cur_max = evt_minmax[v]
                    evt_minmax[v] = (
                        vmin if cur_min is None else min(cur_min, vmin),
                        vmax if cur_max is None else max(cur_max, vmax),
                    )

                    # sample for quantiles
                    take = min(SAMPLE_PER_CHUNK, len(s_non))
                    if take > 0:
                        evt_sample[v].extend(s_non.sample(n=take, random_state=42).astype("float64").tolist())

        # Optimize dtypes & reorder
        chunk = optimize_dtypes(chunk)
        chunk = reorder_columns(chunk)

        # Partition write
        write_city_parts(chunk, TMP_PARTS_DIR, part_counter)

        total_rows_out += len(chunk)

        del shifted, chunk
        gc.collect()

        if i % 5 == 0:
            print(f"  processed rows so far: {total_rows_in:,}")

    # Determine globally 100% null columns (rule-based)
    all_null_cols = sorted([c for c, nn in nonnull_counts.items() if nn == 0])

    print("\n=== Phase 2: Per-city sort + diagnostics (duplicates, max gap) ===")
    city_dirs = sorted(TMP_PARTS_DIR.glob("city_name=*"), key=lambda p: p.name)
    sorted_city_files: List[Path] = []

    # City diagnostics computed on sorted per-city data (robust and actionable)
    dup_counts: Dict[str, int] = {}
    dup_rates: Dict[str, float] = {}
    max_gap_days: Dict[str, Optional[int]] = {}

    for city_dir in city_dirs:
        city_name = city_dir.name.split("=", 1)[1]

        dataset = ds.dataset(str(city_dir), format="parquet")
        table = dataset.to_table()
        df_city = table.to_pandas()

        # Drop globally 100% null columns
        drop_cols = [c for c in all_null_cols if c in df_city.columns]
        if drop_cols:
            df_city.drop(columns=drop_cols, inplace=True)

        # Sort by fecha_local (within city)
        df_city.sort_values(["fecha_local"], ascending=True, inplace=True, kind="mergesort")

        # --- Diagnostics ---
        # Duplicate timestamps within city (fecha_local)
        if "fecha_local" in df_city.columns:
            dups = int(df_city["fecha_local"].duplicated(keep=False).sum())
            dup_counts[city_name] = dups
            dup_rates[city_name] = (dups / len(df_city) * 100.0) if len(df_city) else 0.0

            # Max gap in days between consecutive timestamps (ignoring duplicates)
            # Use unique sorted timestamps for gap computation
            ts = df_city["fecha_local"].dropna().drop_duplicates().sort_values()
            if len(ts) >= 2:
                gaps = ts.diff().dropna()
                # convert to days
                max_gap = int(np.ceil(gaps.max() / np.timedelta64(1, "D")))
                max_gap_days[city_name] = max_gap
            else:
                max_gap_days[city_name] = None

        # Re-optimize and reorder after concatenation
        df_city = optimize_dtypes(df_city)
        df_city = reorder_columns(df_city)

        out_city = TMP_SORTED_DIR / f"{city_name}.parquet"
        pq.write_table(pa.Table.from_pandas(df_city, preserve_index=False), out_city, compression="snappy")
        sorted_city_files.append(out_city)

        del table, df_city
        gc.collect()

        print(f"  sorted city: {city_name}")

    print("\n=== Phase 3: Merge sorted city files into single Parquet (Snappy) ===")
    if out_parquet.exists():
        out_parquet.unlink()

    writer = None
    try:
        for city_file in sorted(sorted_city_files, key=lambda p: p.stem):
            t = pq.read_table(city_file)
            if writer is None:
                writer = pq.ParquetWriter(out_parquet, t.schema, compression="snappy")
            writer.write_table(t)
        print(f"✅ Wrote: {out_parquet}")
    finally:
        if writer is not None:
            writer.close()

    # =============================================================================
    # FINAL REPORT
    # =============================================================================
    print("\n=== VALIDATION REPORT ===")
    print(f"Input rows:  {total_rows_in:,}")
    print(f"Output rows: {total_rows_out:,}")
    print("✅ No observation loss" if total_rows_in == total_rows_out else "⚠️ Row count mismatch!")

    print(f"\nDatetime parse failures (dt_iso -> dt_utc): {dt_parse_fail_count:,}")
    print(f"Timezone mismatches (shifted - dt_utc != timezone): {tz_mismatch_count:,}")

    print("\nLocal-time range (fecha_local, naive local time):")
    print(f"  min: {min_fecha_local}")
    print(f"  max: {max_fecha_local}")

    print("\nGlobally 100% null columns detected (dropped):")
    print(all_null_cols if all_null_cols else "(none)")

    # Missingness (top 15)
    miss_df = pd.DataFrame({
        "column": list(null_counts.keys()),
        "missing_count": list(null_counts.values()),
        "missing_pct": [100.0 * null_counts[c] / total_rows_in for c in null_counts.keys()],
    }).sort_values("missing_pct", ascending=False)

    print("\nMost-missing columns (top 15):")
    print(miss_df.head(15).to_string(index=False))

    print("\nCity counts:")
    for c in sorted(city_counts.keys()):
        print(f"  {c}: {city_counts[c]:,}")

    # Timezone distribution per city (compact)
    print("\nTimezone distribution per city (seconds offset):")
    for c in sorted(timezone_counts.keys()):
        total = sum(timezone_counts[c].values())
        parts = []
        for tz in sorted(timezone_counts[c].keys()):
            cnt = timezone_counts[c][tz]
            parts.append(f"{tz}:{cnt} ({cnt/total*100:.2f}%)")
        print(f"  {c}: " + ", ".join(parts))

    # Duplicate timestamps + max gap diagnostics
    print("\nDuplicate timestamps per city (fecha_local):")
    for c in sorted(dup_counts.keys()):
        print(f"  {c}: {dup_counts[c]:,}  ({dup_rates[c]:.4f}%)")

    print("\nMax gap between unique timestamps per city (days):")
    for c in sorted(max_gap_days.keys()):
        print(f"  {c}: {max_gap_days[c]}")

    # EVT preview (approx quantiles from samples)
    print("\nEVT preview (non-destructive):")
    for v in EVT_PREVIEW_FLOATS:
        if evt_minmax[v][0] is None:
            continue
        q = safe_quantiles(pd.Series(evt_sample[v], dtype="float64"), EVT_PREVIEW_Q)
        print(f"  {v}: min={evt_minmax[v][0]:.4g}, max={evt_minmax[v][1]:.4g}, "
              + ", ".join([f"q{int(qq*100)}={q[qq]:.4g}" for qq in q]))

    # Final schema
    try:
        schema = pq.read_schema(out_parquet)
        print("\nFinal Parquet schema:")
        print(schema)
    except Exception as e:
        print(f"\n(Info) Could not read final schema: {e}")

    print("\n=== Done ===")

    # Optional cleanup
    # shutil.rmtree(TMP_PARTS_DIR, ignore_errors=True)
    # shutil.rmtree(TMP_SORTED_DIR, ignore_errors=True)


if __name__ == "__main__":
    clean_evt_pipeline(csv_path=CSV_PATH, out_parquet=OUT_PARQUET, chunksize=CHUNKSIZE)


=== Phase 1: Chunked transform + per-city partition write ===
  processed rows so far: 1,500,000
  processed rows so far: 3,000,000
  processed rows so far: 4,500,000
  processed rows so far: 6,000,000

=== Phase 2: Per-city sort + diagnostics (duplicates, max gap) ===
  sorted city: Ambato
  sorted city: Cuenca
  sorted city: Esmeraldas
  sorted city: Guayaquil
  sorted city: Ibarra
  sorted city: Lago Agrio
  sorted city: Loja
  sorted city: Machala
  sorted city: Manta
  sorted city: Puerto Morona
  sorted city: Puyo
  sorted city: Quevedo
  sorted city: Quito
  sorted city: Santa Cruz Island
  sorted city: Santo Domingo
  sorted city: Zamora

=== Phase 3: Merge sorted city files into single Parquet (Snappy) ===
✅ Wrote: /content/drive/MyDrive/extreme-climate-forecasting/data/data_clima_clean.parquet

=== VALIDATION REPORT ===
Input rows:  6,445,858
Output rows: 6,445,858
✅ No observation loss

Datetime parse failures (dt_iso -> dt_utc): 0
Timezone mismatches (shifted - dt_utc != ti

### Check dataset

In [3]:
# Load the dataset
my_file = 'data_clima_clean.parquet'
file_path = f'/content/drive/MyDrive/extreme-climate-forecasting/data/{my_file}'

try:
    # data = pd.read_csv(file_path)   # dataset name
    data = pd.read_parquet(file_path)
    print(f"Shape: {data.shape}")
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")

Shape: (6445858, 30)


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6445858 entries, 0 to 6445857
Data columns (total 30 columns):
 #   Column               Dtype              
---  ------               -----              
 0   city_name            category           
 1   dt                   int32              
 2   dt_iso               object             
 3   dt_utc               datetime64[ns, UTC]
 4   timezone             int16              
 5   fecha_local          datetime64[ns]     
 6   year                 int32              
 7   month                int32              
 8   day                  int32              
 9   hour                 int32              
 10  lat                  float32            
 11  lon                  float32            
 12  temp                 float32            
 13  visibility           float32            
 14  dew_point            float32            
 15  feels_like           float32            
 16  temp_min             float32            
 17  temp_max

In [5]:
data # head and tail

,city_name,dt,dt_iso,dt_utc,timezone,fecha_local,year,month,day,hour,...,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,Ambato,283996800,1979-01-01 00:00:00 +0000 UTC,1979-01-01 00:00:00+00:00,-18000,1978-12-31 19:00:00,1978,12,31,19,...,1.88,112,NaN,NaN,NaN,44,802,Clouds,scattered clouds,03n
1,Ambato,284000400,1979-01-01 01:00:00 +0000 UTC,1979-01-01 01:00:00+00:00,-18000,1978-12-31 20:00:00,1978,12,31,20,...,1.68,101,NaN,NaN,NaN,92,804,Clouds,overcast clouds,04n
2,Ambato,284004000,1979-01-01 02:00:00 +0000 UTC,1979-01-01 02:00:00+00:00,-18000,1978-12-31 21:00:00,1978,12,31,21,...,1.51,103,NaN,NaN,NaN,93,804,Clouds,overcast clouds,04n
3,Ambato,284007600,1979-01-01 03:00:00 +0000 UTC,1979-01-01 03:00:00+00:00,-18000,1978-12-31 22:00:00,1978,12,31,22,...,1.47,107,NaN,NaN,NaN,93,804,Clouds,overcast clouds,04n
4,Ambato,284011200,1979-01-01 04:00:00 +0000 UTC,1979-01-01 04:00:00+00:00,-18000,1978-12-31 23:00:00,1978,12,31,23,...,1.45,111,NaN,NaN,NaN,95,804,Clouds,overcast clouds,04n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6445853,Zamora,1729364400,2024-10-19 19:00:00 +0000 UTC,2024-10-19 19:00:00+00:00,-18000,2024-10-19 14:00:00,2024,10,19,14,...,2.64,88,1.80,NaN,NaN,55,803,Clouds,broken clouds,04d
6445854,Zamora,1729368000,2024-10-19 20:00:00 +0000 UTC,2024-10-19 20:00:00+00:00,-18000,2024-10-19 15:00:00,2024,10,19,15,...,2.36,58,3.01,NaN,NaN,54,803,Clouds,broken clouds,04d
6445855,Zamora,1729371600,2024-10-19 21:00:00 +0000 UTC,2024-10-19 21:00:00+00:00,-18000,2024-10-19 16:00:00,2024,10,19,16,...,2.36,58,3.01,NaN,NaN,54,803,Clouds,broken clouds,04d
6445856,Zamora,1729375200,2024-10-19 22:00:00 +0000 UTC,2024-10-19 22:00:00+00:00,-18000,2024-10-19 17:00:00,2024,10,19,17,...,2.36,58,3.01,NaN,NaN,54,803,Clouds,broken clouds,04d


In [6]:
data.describe()    # Get summary statistics

,dt,timezone,fecha_local,year,month,day,hour,lat,lon,temp,...,temp_max,pressure,humidity,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,clouds_all,weather_id
count,6.445858e+06,6.445858e+06,6445858,6.445858e+06,6.445858e+06,6.445858e+06,6.445858e+06,6.445858e+06,6.445858e+06,6.445858e+06,...,6.445858e+06,6.445858e+06,6.445858e+06,6.445858e+06,6.445858e+06,56466.000000,2.311264e+06,1498.000000,6.445858e+06,6.445858e+06
mean,1.007273e+09,-1.817513e+04,2001-12-02 00:58:41.613666688,2.001420e+03,6.499152e+00,1.572299e+01,1.150095e+01,-1.480567e+00,-7.963538e+01,2.110215e+01,...,2.206570e+01,1.013676e+03,8.419744e+01,1.924652e+00,1.818743e+02,2.135571,6.772044e-01,1.307477,7.948018e+01,6.910056e+02
min,2.839968e+08,-2.160000e+04,1978-12-31 19:00:00,1.978000e+03,1.000000e+00,1.000000e+00,0.000000e+00,-4.062094e+00,-9.033719e+01,1.900000e-01,...,1.990000e+00,1.001000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000e-01,0.130000,0.000000e+00,2.000000e+02
25%,6.460164e+08,-1.800000e+04,1990-06-21 20:00:00,1.990000e+03,4.000000e+00,8.000000e+00,6.000000e+00,-2.882745e+00,-7.965300e+01,1.758000e+01,...,1.885000e+01,1.011000e+03,7.800000e+01,8.900000e-01,1.010000e+02,0.000000,1.800000e-01,0.645000,6.900000e+01,5.000000e+02
50%,1.007658e+09,-1.800000e+04,2001-12-06 12:00:00,2.001000e+03,7.000000e+00,1.600000e+01,1.200000e+01,-1.022512e+00,-7.900590e+01,2.194000e+01,...,2.274000e+01,1.013000e+03,8.700000e+01,1.500000e+00,1.840000e+02,0.000000,3.400000e-01,1.000000,9.200000e+01,8.020000e+02
75%,1.368598e+09,-1.800000e+04,2013-05-15 01:00:00,2.013000e+03,9.000000e+00,2.300000e+01,1.700000e+01,-2.232520e-01,-7.851411e+01,2.465000e+01,...,2.537000e+01,1.016000e+03,9.300000e+01,2.600000e+00,2.610000e+02,3.130000,7.600000e-01,1.130000,9.900000e+01,8.040000e+02
max,1.729379e+09,-1.440000e+04,2024-10-19 18:00:00,2.024000e+03,1.200000e+01,3.100000e+01,2.300000e+01,9.705810e-01,-7.691194e+01,3.893000e+01,...,3.946000e+01,1.029000e+03,1.000000e+02,8.624000e+01,3.600000e+02,149.199997,4.877000e+01,49.000000,1.000000e+02,8.040000e+02
std,4.172152e+08,8.365979e+02,NaN,1.322073e+01,3.443060e+00,8.798920e+00,6.920700e+00,1.538459e+00,5.083071e+00,4.984807e+00,...,4.784522e+00,3.627487e+00,1.224997e+01,1.469900e+00,9.260996e+01,3.655935,1.018558e+00,2.128318,2.551184e+01,1.461824e+02


### Unique values

In [7]:
# Checks unique values in every column
for col in data.columns:
    unique_values = data[col].unique()
    print(f"Column '{col}':")
    print(unique_values)
    print(f"Number of unique values: {len(unique_values)}")
    print("-" * 60)
    print()

Column 'city_name':
['Ambato', 'Cuenca', 'Esmeraldas', 'Guayaquil', 'Ibarra', ..., 'Quevedo', 'Quito', 'Santa Cruz Island', 'Santo Domingo', 'Zamora']
Length: 16
Categories (16, object): ['Ambato', 'Cuenca', 'Quito', 'Santo Domingo', ..., 'Manta', 'Quevedo',
                          'Puerto Morona', 'Santa Cruz Island']
Number of unique values: 16
------------------------------------------------------------

Column 'dt':
[ 283996800  284000400  284004000 ... 1729371600 1729375200 1729378800]
Number of unique values: 401496
------------------------------------------------------------

Column 'dt_iso':
['1979-01-01 00:00:00 +0000 UTC' '1979-01-01 01:00:00 +0000 UTC'
 '1979-01-01 02:00:00 +0000 UTC' ... '2024-10-19 21:00:00 +0000 UTC'
 '2024-10-19 22:00:00 +0000 UTC' '2024-10-19 23:00:00 +0000 UTC']
Number of unique values: 401496
------------------------------------------------------------

Column 'dt_utc':
<DatetimeArray>
['1979-01-01 00:00:00+00:00', '1979-01-01 01:00:00+00:00',
 '197

### Missing values

In [8]:
# Analyze missing values
missing_values = data.isnull().sum()
missing_percentage = (missing_values / len(data)* 100).round(4)

# Display columns with missing values and their percentages
missing_data_summary = missing_values[missing_values > 0].to_frame(name="Missing Count")
missing_data_summary["Missing Percentage"] = missing_percentage[missing_values > 0]
missing_data_summary

,Missing Count,Missing Percentage
visibility,5409494,83.9220
wind_gust,6389392,99.1240
rain_1h,4134594,64.1434
rain_3h,6444360,99.9768
